In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[3]:


import requests
import re
import pandas as pd
# from openpyxl import Workbook

# 72987 百万二
drama_id = 'https://www.missevan.com/mdrama/72987'  #输入剧集地址

drama_url = 'https://www.missevan.com/dramaapi/getdrama?drama_id=   '
url = drama_url + drama_id.split("/")[4]
drama_result = requests.get(url).text
pattern1 = re.compile('sound_id":(.+?),"type')
pattern2 = re.compile('"need_pay":(.+?),"')
sound_id = re.findall(pattern1, drama_result)
pay_type = re.findall(pattern2, drama_result)
pay_type = pay_type[1:]
drama = pd.DataFrame({'sound_id': sound_id, 'pay_type': pay_type})
sound_id = list(drama['sound_id'][drama['pay_type'] == '1'])
sound_url = 'https://www.missevan.com/sound/getdm?soundid='
for i in range(0, len(sound_id)):
    url = sound_url + sound_id[i]
    sound_result = requests.get(url).text
    pattern3 = re.compile(r'p="(.+?)"')
    dm = re.findall(pattern3, sound_result)

    if i == 0:
        dm1 = pd.DataFrame({'dm': dm})
        dms = dm1['dm'].str.split('>', expand=True)[0]
        uid = dms.str.split(',', expand=True)[6]
        ctime = dms.str.split(',', expand=True)[4]
        dm1['uid'] = uid
        dm1['ctime'] = ctime.astype(int)
        dm1 = dm1[dm1['ctime'] > 1709977800]  # 筛选出非限免期间的用户
    else:
        dm2 = pd.DataFrame({'dm': dm})
        dms_i = dm2['dm'].str.split('>', expand=True)[0]
        uid_i = dms_i.str.split(',', expand=True)[6]
        dm2['uid'] = uid_i
        dm1 = dm1._append(dm2)
print("本剧实时付费ID为:", len(dm1['uid'].unique()))
print("本剧实时付费弹幕数量为:", len(dm1['uid']))
#
# dmDis = dm1['uid'].unique()
# # print(len(dmDis))
# # 创建一个新的工作簿
# wb = Workbook()
# # 选择默认的工作表
# ws = wb.active
# # 数据可以直接分配到单元格中
# ws['A1'] = "付费id"
# # 可以使用append方法插入一行数据
# for k in range(0, len(dmDis)):
#     ws.append([dmDis[k]])
# # 保存工作簿到文件
# wb.save("百万二付费ID明细.xlsx")
# print("百万二的付费ID明细写入Excel完成！！！")
# In[ ]:
